# Optimize Forecast and Thresholding Model

In [1]:
!pip install calflops lightning gdown numba pyts==0.12.0
!pip install --no-deps tsai==0.3.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 12.7 MB/s eta 0:00:00


In [2]:
import os
import yaml
import json
import logging
import warnings
import shutil
import gdown
import optuna
from optuna.samplers import TPESampler
from optuna.exceptions import OptunaError
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from numba import njit
from torch.utils.data import DataLoader, SubsetRandomSampler, Dataset
from calflops import calculate_flops
import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger, Logger
from lightning.pytorch.utilities import disable_possible_user_warnings, rank_zero_only
from fastai.imports import noop
from fastai.layers import AdaptiveConcatPool1d
from tsai.models.layers import Conv, Concat, Norm, ConvBlock, GAP1d, Add
from typing import Literal
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor


logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
disable_possible_user_warnings()


class Config:
    """Loads parameters from config.yaml into global object"""
    def __init__(self, path_to_config=None):

        if path_to_config:
            self.path_to_config = path_to_config

            if os.path.isfile(path_to_config):
                pass
            else:
                self.path_to_config = '../{}'.format(self.path_to_config)

            with open(self.path_to_config, "r") as f:
                self.dictionary = yaml.load(f.read(), Loader=yaml.FullLoader)

            for k, v in self.dictionary.items():
                setattr(self, k, v)

    @staticmethod
    def from_dict(d: dict) -> 'Config':
        c = Config()
        c.dictionary = d
        for k, v in c.dictionary.items():
            setattr(c, k, v)
        return c

    def __setitem__(self, key, value):
        self.dictionary[key] = value
        setattr(self, key, value)

    @staticmethod
    def build_group_lookup(path_to_groupings):
        channel_group_lookup = {}

        with open(path_to_groupings, "r") as f:
            groupings = json.loads(f.read())

            for subsystem in groupings.keys():
                for subgroup in groupings[subsystem].keys():
                    for chan in groupings[subsystem][subgroup]:
                        channel_group_lookup[chan["key"]] = {}
                        channel_group_lookup[chan["key"]]["subsystem"] = subsystem
                        channel_group_lookup[chan["key"]]["subgroup"] = subgroup

        return channel_group_lookup


class BaseLightningModel(L.LightningModule):
    def __init__(self, config=None):
        super().__init__()
        self.config = config
        self._val_loss = 0.0
        self._test_loss = 0.0
        self._val_batches = 0
        self._test_batches = 0

    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        loss = self.criterion(output, target)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        self._val_loss += self.val_criterion(output, target).item()
        self._val_batches += 1

    def test_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        self._test_loss += self.val_criterion(output, target).item()
        self._test_batches += 1

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if isinstance(batch, list) or isinstance(batch, tuple):
            batch, y = batch
            return self(batch), y
        return self(batch)

    def configure_optimizers(self):
        if self.config.optimizer == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=self.config.lr)
        elif self.config.optimizer == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.lr)
        else:
            raise ValueError(f"Unknown optimizer: {self.config.optimizer}")
        if self.config.use_one_cycle:
            scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=self.config.lr, pct_start=0.25,
                                                            total_steps=self.trainer.estimated_stepping_batches)
            opt_config = {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": scheduler,
                    "interval": "step",
                    "frequency": 1,
                }
            }
            return opt_config
        return optimizer

    def configure_callbacks(self):
        callbacks = []
        early_stop = EarlyStopping(
            monitor="val_loss",
            patience=self.config.patience,
            min_delta=self.config.min_delta,
            verbose=True
        )
        callbacks.append(early_stop)
        if self.config.modelOutput is not None:
            checkpoint = ModelCheckpoint(
                dirpath=self.config.modelOutput,
                filename="best_{epoch:02d}-{val_loss:.3f}",
                monitor="val_loss",
                save_top_k=1,
                verbose=True,
                save_last=True
            )
            callbacks.append(checkpoint)
        lr_monitor = LearningRateMonitor(logging_interval="step")
        callbacks.append(lr_monitor)
        return callbacks

    def on_validation_epoch_end(self):
        """Compute and log validation metrics at the end of the validation epoch."""
        val_loss = self._val_loss / self._val_batches
        self.log("val_loss", val_loss)
        self._val_loss = 0.0
        self._val_batches = 0

    def on_test_epoch_end(self):
        """Compute and log test metrics at the end of the test epoch."""
        test_loss = self._test_loss / self._test_batches
        self.log("test_loss", test_loss)
        self._test_loss = 0.0
        self._test_batches = 0

    @staticmethod
    def get_act(activation: Literal["relu", "leakyrelu", "mish", "silu", "hardswish", "gelu", "celu", "elu"] = "relu"):
        activation = activation.lower()
        if activation == "relu":
            return torch.nn.ReLU
        elif activation == "leakyrelu":
            return torch.nn.LeakyReLU
        elif activation == "mish":
            return torch.nn.Mish
        elif activation == "silu":  # Swish
            return torch.nn.SiLU
        elif activation == "hardswish":
            return torch.nn.Hardswish
        elif activation == "gelu":
            return torch.nn.GELU
        elif activation == "celu":
            return torch.nn.CELU
        elif activation == "elu":
            return torch.nn.ELU
        else:
            raise ValueError(f"Unknown activation function: {activation}")


class BaseForecaster(BaseLightningModel):
    def __init__(self, config):
        super().__init__(config)
        self.criterion = self.get_loss_metric(self.config.loss_metric)
        self.val_criterion = self.get_loss_metric(self.config.loss_metric)

        self.save_hyperparameters(config.dictionary)

    @staticmethod
    def get_loss_metric(loss_metric: str):
        loss_metric = loss_metric.lower()
        if loss_metric == "mse":
            return torch.nn.MSELoss()
        elif loss_metric == "mae":
            return torch.nn.L1Loss()
        elif loss_metric == "huber":
            return torch.nn.HuberLoss()
        else:
            raise ValueError(f"Unknown loss metric: {loss_metric}")


class XceptionModulePlus(nn.Module):
    def __init__(self, ni, nf, ks=40, kss=None, bottleneck=True, coord=False, separable=True, norm='Batch',
                 bn_1st=True, act=nn.ReLU, act_kwargs=None, norm_act=False):
        super().__init__()
        act_kwargs = {} if act_kwargs is None else act_kwargs
        if kss is None:
            kss = [ks // (2 ** i) for i in range(3)]
        kss = [ksi if ksi % 2 != 0 else ksi - 1 for ksi in kss]  # ensure odd kss for padding='same'
        self.bottleneck = Conv(ni, nf, 1, coord=coord, bias=False) if bottleneck else noop
        self.convs = nn.ModuleList()
        for i in range(len(kss)):
            self.convs.append(Conv(nf if bottleneck else ni, nf, kss[i], coord=coord, separable=separable, bias=False))
        self.mp_conv = nn.Sequential(*[nn.MaxPool1d(3, stride=1, padding=1), Conv(ni, nf, 1, coord=coord, bias=False)])
        self.concat = Concat()
        _norm_act = []
        if act is not None:
            _norm_act.append(act(**act_kwargs))
        _norm_act.append(Norm(nf * 4, norm=norm, zero_norm=False))
        if bn_1st:
            _norm_act.reverse()
        self.norm_act = noop if not norm_act else _norm_act[0] if act is None else nn.Sequential(*_norm_act)

    def forward(self, x):
        input_tensor = x
        x = self.bottleneck(x)
        x = self.concat([l(x) for l in self.convs] + [self.mp_conv(input_tensor)])
        return self.norm_act(x)


class XceptionBlockPlus(nn.Module):
    def __init__(self, ni, nf, residual=True, coord=False, norm='Batch', act=nn.ReLU, act_kwargs=None, dropout=0., **kwargs):
        super().__init__()
        act_kwargs = {} if act_kwargs is None else act_kwargs
        self.residual = residual
        self.xception, self.shortcut, self.act = nn.ModuleList(), nn.ModuleList(), nn.ModuleList()
        for i in range(4):
            if self.residual and (i - 1) % 2 == 0:
                self.shortcut.append(
                    Norm(n_in, norm=norm) if n_in == n_out else
                    ConvBlock(n_in, n_out * 4 * 2, 1, coord=coord, bias=False, norm=norm, act=None, dropout=dropout)
                )
                self.act.append(act(**act_kwargs))
            n_out = nf * 2 ** i
            n_in = ni if i == 0 else n_out * 2
            self.xception.append(XceptionModulePlus(n_in, n_out, coord=coord, norm=norm,
                                                    act=act if self.residual and (i - 1) % 2 == 0 else None, **kwargs))
        self.add = Add()

    def forward(self, x):
        res = x
        for i in range(4):
            x = self.xception[i](x)
            if self.residual and (i + 1) % 2 == 0:
                res = x = self.act[i // 2](self.add(x, self.shortcut[i // 2](res)))
        return x


class XceptionTimePlus(BaseForecaster):
    def __init__(self, config):
        super().__init__(config)

        # params
        c_in = len(config.input_channels)
        c_out = len(config.target_channels) * config.prediction_window_size
        nf = config.nf  # 16
        coord = config.coord  # False
        norm = config.norm  # 'Batch'
        concat_pool = config.concat_pool  # False
        adaptive_size = config.adaptive_size  # 50
        act = self.get_act(config.activation)  # nn.ReLU
        dropout = config.dropout  # 0.
        ks = config.ks  # 40
        bottleneck = config.bottleneck  # True
        bn_1st = config.bn_1st  # True
        norm_act = config.norm_act  # False
        width = config.width  # 16

        self.prediction_window_size = config.prediction_window_size
        self.target_channels = len(config.target_channels)

        # Backbone
        self.backbone = XceptionBlockPlus(c_in, nf, coord=coord, norm=norm, act=act, dropout=dropout, ks=ks,
                                          bottleneck=bottleneck, bn_1st=bn_1st, norm_act=norm_act)
        # Head
        gap1 = AdaptiveConcatPool1d(adaptive_size) if adaptive_size and concat_pool else nn.AdaptiveAvgPool1d(adaptive_size) if adaptive_size else noop
        mult = 2 if adaptive_size and concat_pool else 1
        conv1x1_1 = ConvBlock(nf * 32 * mult, nf * width * mult, 1, coord=coord, norm=norm)
        conv1x1_2 = ConvBlock(nf * width * mult, nf * width // 2 * mult, 1, coord=coord, norm=norm)
        conv1x1_3 = ConvBlock(nf * width // 2 * mult, c_out, 1, coord=coord, norm=norm)
        gap2 = GAP1d(1)
        lin = nn.Linear(c_out, c_out)  # Added by me to avoid ReLU preventing negative values
        self.head = nn.Sequential(gap1, conv1x1_1, conv1x1_2, conv1x1_3, gap2, lin)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.backbone(x)
        x = self.head(x)
        x = x.view(-1, self.prediction_window_size, self.target_channels)
        return x


class TSForecastDataset(Dataset):
    def __init__(
            self,
            X: np.ndarray,
            window_size: int = 1,
            stride: int = 1,
            pred_horizon: int = 1,
            channels_first: bool = False,
            return_y: bool = True
    ):

        self.window_size = window_size
        self.stride = stride
        self.pred_horizon = pred_horizon
        self.channels_first = channels_first
        self.return_y = return_y
        self.number_of_windows = ((len(X) - window_size - pred_horizon) // stride) + 1
        self.X = torch.tensor(X, dtype=torch.float32)

    def __len__(self):
        return self.number_of_windows

    def __getitem__(self, idx):
        # get window
        start_idx = idx * self.stride
        end_idx = start_idx + self.window_size

        window = self.X[start_idx:end_idx]  # (window_size, n_features)

        # change to channels first format
        if self.channels_first:
            window = window.permute(1, 0)

        # get label and return
        if self.return_y:
            y_end_idx = end_idx + self.pred_horizon
            return window, self.X[end_idx:y_end_idx]
        return window


class InternalLogger(Logger):
    """Internal logger for storing metrics in memory."""
    def __init__(self):
        super().__init__()
        self.metrics = []
        self.hyper_params = {}

    @property
    def history(self):
        """Get metrics history as a DataFrame."""
        metric_df = pd.DataFrame(self.metrics)
        metric_df = metric_df.groupby("step").first().reset_index()
        return metric_df

    @property
    def name(self):
        return "InternalLogger"

    @property
    def version(self):
        return "1.0"

    @rank_zero_only
    def log_hyperparams(self, params):
        self.hyper_params = params

    @rank_zero_only
    def log_metrics(self, metrics, step):
        metrics["step"] = step
        self.metrics.append(metrics)

    @rank_zero_only
    def save(self):
        pass

    @rank_zero_only
    def save_to_csv(self, path: str):
        """Save metrics to a CSV file."""
        metric_df = self.history
        metric_df.to_csv(path)
        print(f"Metrics saved to {path}")

    @rank_zero_only
    def finalize(self, status):
        pass


class TrialError(OptunaError):
    pass


def prepare_forecast_sampler(y: np.ndarray, num_samples: int, window_size: int = 250, stride: int = 1, drop_last: int = 1):
    # Extract labels from the training dataset
    global_labels = y.max(axis=1).astype(int)
    global_labels = global_labels[window_size - 1::stride]
    # Identify indices for nominal windows
    idx_nominal = np.where(global_labels == 0)[0]
    # drop last idx since no future labels exist
    idx_nominal = idx_nominal[:-drop_last]
    # select num_samples randomly
    idx_nominal = np.random.choice(idx_nominal, num_samples, replace=False)
    # Create a sampler for nominal data
    sampler = SubsetRandomSampler(idx_nominal)
    return sampler


@njit
def expand_labels(labels: np.ndarray, forward_extension: int = 1, backward_extension: int = 0) -> np.ndarray:
    """Expands positive labels in a 2D binary label array along the time axis."""
    if labels.ndim != 2:
        raise ValueError("Input array must be 2D with shape (n_samples, n_multi_labels).")

    # Perform label expansion using numba-accelerated function
    n_samples, n_labels = labels.shape
    extended_labels = labels.copy()
    for j in range(n_labels):
        for i in range(n_samples):
            if labels[i, j] == 1:
                start = i - backward_extension
                end = i + forward_extension + 1  # +1 to include the endpoint
                if start < 0:
                    start = 0
                if end > n_samples:
                    end = n_samples
                for k in range(start, end):
                    extended_labels[k, j] = 1

    return extended_labels

In [3]:
macs_threshold = 70  # Million MACs
params_threshold = 350_000  # Num Params
n_startup_trials = 20
num_samples_per_epoch = 2_097_152  # 2048 steps
num_samples_per_val_epoch = 262_144  # 256 steps
batch_size = 1024
accumulate_grad_batches = 2048 // batch_size  # virtual batch size of 2048
num_evals = 4  # number of evaluations per epoch
split = 264_960
file_dir = "/kaggle/working"
study_name = "model_optimization_ft"
_id = "1-_Ka_AcSqvGPngoVNqcpjovzqhcK6p4O" # Google Drive file id
file_name = "ft_run_04_output.zip"  # zip file to download

seed = 0
stride = 1
in_channels = 6
out_channels = 6
eval_steps = num_samples_per_epoch // batch_size // num_evals
url = f"https://drive.google.com/uc?id={_id}"
output_file = f"{file_dir}/{file_name}"

In [4]:
# download zip
gdown.download(url, output_file, quiet=False)
# unpack previous run output
shutil.unpack_archive(output_file, file_dir)
# clean up zip file
os.remove(output_file)

Downloading...
From: https://drive.google.com/uc?id=1-_Ka_AcSqvGPngoVNqcpjovzqhcK6p4O
To: /kaggle/working/ft_run_04_output.zip
100%|██████████| 2.26M/2.26M [00:00<00:00, 180MB/s]


In [5]:
df_train = pd.read_csv("/kaggle/input/esa-mission-1-train-dataset/84_months.train.csv")
target_channels = [f"channel_{i}" for i in range(41, 47)]
label_cols = ["is_anomaly_" + col for col in target_channels]

train_array = df_train[target_channels].iloc[:-split].values.astype(np.float32)
val_array = df_train[target_channels].iloc[-split:].values.astype(np.float32)
train_labels = df_train[label_cols].iloc[:-split].values.astype(np.float32).clip(0., 1.)
val_labels = df_train[label_cols].iloc[-split:].values.astype(np.float32).clip(0., 1.)

# standard scaling
train_means = np.mean(train_array, axis=0)
train_stds = np.std(train_array, axis=0)
train_array_scaled = (train_array - train_means) / train_stds  # + eps
val_array_scaled = (val_array - train_means) / train_stds  # + eps

train_array_scaled.shape, train_labels.shape, val_array_scaled.shape, val_labels.shape

((7099201, 6), (7099201, 6), (264960, 6), (264960, 6))

In [6]:
train_labels = expand_labels(train_labels, 260, 20)  # max lookback 260, max fct horizon 20
val_labels = expand_labels(val_labels, 260, 20)  # max lookback 260, max fct horizon 20

In [7]:
def objective(trial):
    # sample model configuration
    window_size = trial.suggest_int("window_size", 16, 256, step=8)
    pred_horizon = trial.suggest_int("pred_horizon", 1, 10, step=1)
    nf = trial.suggest_int("nf", 4, 32, step=4)
    coord = trial.suggest_categorical("coord", [True, False])
    norm = trial.suggest_categorical("norm", ["Batch", "Instance"])
    concat_pool = trial.suggest_categorical("concat_pool", [True, False])
    adaptive_size = trial.suggest_int("adaptive_size", 2, 64, step=2)
    activation = trial.suggest_categorical("activation", ["relu", "leakyrelu", "hardswish"])
    dropout = trial.suggest_float("dropout", 0., 0.3, step=0.05)
    ks = trial.suggest_int("ks", 8, 64, step=8)
    bottleneck = trial.suggest_categorical("bottleneck", [True, False])
    bn_1st = trial.suggest_categorical("bn_1st", [True, False])
    norm_act = trial.suggest_categorical("norm_act", [True, False])
    width = trial.suggest_int("width", 4, 16, step=2)

    config = Config.from_dict(
        {
            "input_channels": target_channels,
            "target_channels": target_channels,
            "prediction_window_size": pred_horizon,
            "nf": nf,
            "coord": coord,
            "norm": norm,
            "concat_pool": concat_pool,
            "adaptive_size": adaptive_size,
            "activation": activation,
            "dropout": dropout,
            "ks": ks,
            "bottleneck": bottleneck,
            "bn_1st": bn_1st,
            "norm_act": norm_act,
            "width": width,
            # fixed base params
            "loss_metric": "mse",
            "optimizer": "adam",
            "lr": 2e-4,
            "use_one_cycle": True,
            "patience": 100,
            "min_delta": 0.0,
            "modelOutput": None,
        }
    )

    # set seed
    L.seed_everything(seed=seed, verbose=False)

    # setup datasets
    train_dataset = TSForecastDataset(train_array_scaled, window_size, stride, pred_horizon)
    val_dataset = TSForecastDataset(val_array_scaled, window_size, stride, pred_horizon)

    train_sampler = prepare_forecast_sampler(train_labels, num_samples_per_epoch, window_size, stride, pred_horizon)
    val_sampler = prepare_forecast_sampler(val_labels, num_samples_per_val_epoch, window_size, stride, pred_horizon)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=False, sampler=train_sampler)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False, sampler=val_sampler)

    try:
        # initialize model
        model = XceptionTimePlus(config).cuda()

        # calculate MACs and model parameters
        flops, macs, num_params = calculate_flops(
            model=model,
            input_shape=(1, window_size, in_channels),
            print_results=False,
            output_as_string=False,
            include_backPropagation=False,
        )
        # store all metrics
        trial.set_user_attr("flops", flops)
        trial.set_user_attr("macs", macs)
        trial.set_user_attr("num_params", num_params)

        # scale macs
        macs /= 1_000_000

    except Exception as e:
        raise TrialError(f"Trial {trial.number} model initialization failed: {e}")

    # prune
    # if trial.number >= n_startup_trials:  # Don't prune during startup
    if (macs > macs_threshold) or (num_params > params_threshold):
        raise optuna.TrialPruned()

    # Setup trainer
    metric_logger = InternalLogger()
    # torch.set_float32_matmul_precision("high")

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        trainer = L.Trainer(
            accelerator="gpu",
            max_epochs=1,
            log_every_n_steps=8,
            accumulate_grad_batches=accumulate_grad_batches,
            logger=[metric_logger, TensorBoardLogger(save_dir=file_dir, name="lightning_logs")],
            val_check_interval=eval_steps,
            enable_model_summary=False,
            enable_progress_bar=False,
            enable_checkpointing=False,
        )

    try:
        # train model
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        # get min validation loss
        metric_df = metric_logger.history
        loss = metric_df["val_loss"].min()
        trial.set_user_attr("val_loss", loss)

    except Exception as e:
        raise TrialError(f"Trial {trial.number} training failed: {e}")

    if (loss is None) or (np.isnan(loss)):
        raise TrialError("No validation loss")
    if (macs is None) or (np.isnan(macs)):
        raise TrialError("No MACs")

    return macs, loss

In [8]:
sampler = TPESampler(n_startup_trials=n_startup_trials, multivariate=True, seed=seed)
study = optuna.create_study(
    storage=f"sqlite:///{file_dir}/{study_name}.db",
    study_name=study_name,
    sampler=sampler,
    directions=["minimize", "minimize"],
    load_if_exists=True,
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-03-26 11:42:15,848] Using an existing study with name 'model_optimization_ft' instead of creating a new one.


In [9]:
study.optimize(
    objective,
    n_trials=75,
    timeout=int(60 * 60 * 5),
    catch=(TrialError,),
    gc_after_trial=True
)

[I 2025-03-26 11:43:46,117] Trial 37 finished with values: [0.484544, 0.025798408314585686] and parameters: {'window_size': 16, 'pred_horizon': 4, 'nf': 4, 'coord': True, 'norm': 'Batch', 'concat_pool': True, 'adaptive_size': 4, 'activation': 'hardswish', 'dropout': 0.05, 'ks': 48, 'bottleneck': False, 'bn_1st': True, 'norm_act': False, 'width': 8}.
[I 2025-03-26 11:45:46,681] Trial 38 finished with values: [2.361672, 0.025406941771507263] and parameters: {'window_size': 24, 'pred_horizon': 2, 'nf': 8, 'coord': True, 'norm': 'Batch', 'concat_pool': True, 'adaptive_size': 6, 'activation': 'hardswish', 'dropout': 0.05, 'ks': 48, 'bottleneck': False, 'bn_1st': True, 'norm_act': False, 'width': 8}.
[I 2025-03-26 11:47:03,108] Trial 39 finished with values: [0.343144, 0.026380909606814384] and parameters: {'window_size': 16, 'pred_horizon': 3, 'nf': 4, 'coord': True, 'norm': 'Batch', 'concat_pool': True, 'adaptive_size': 2, 'activation': 'hardswish', 'dropout': 0.05, 'ks': 24, 'bottleneck':